# Détection automatique d'erreurs d'OCR

## Imports

In [1]:
import nltk
from thefuzz import fuzz
from thefuzz import process
from Levenshtein import distance

ModuleNotFoundError: No module named 'thefuzz'

Nous allons d'abord utiliser la distance de Levenshtein, selon sa formule originale (1965)

In [ ]:
#difference entre les mots avec la distance de live,shtein (ou truc du genre)
# on peut voir les percentoige de differences entre les mots avec thefuzz
# on va pouvoir utiliser ces dista,ces là même quand il ya par exemple un erreur de frappe
# si on a des chainse plus grandes on peut voir si tout les mots d'un textes peuvent etre retrouvés dans un autre texte
# là on peut donc pne pas prendre en compte l'ordre des mots (plus bas token_sort_ratio)
distance("bulletin", "ballotins")

Nous allons ensuite faire appel à la librairie TheFuzz (anciennement fuzzywuzzy) qui repose sur python-Levenshtein (https://github.com/seatgeek/thefuzz).
Celle-ci permet de normaliser le résultat en calculant un ratio (pourcentage) de similarité entre deux chaînes de caractères :

In [ ]:
fuzz.ratio("bulletin", "ballotins")

La fonction  `partial_ratio` se focalise sur la sous-chaîne (substring) la plus similaire : 

In [ ]:
fuzz.partial_ratio("Grand", "Grand-Place")

La fonction `token_sort_ratio` ignore l'ordre des mots :

In [ ]:
fuzz.token_sort_ratio("archives de la ville de Bruxelles", "de Bruxelles la ville de archives")

La fonction `token_set_ratio` ignore les répétitions de mots :

In [ ]:
# on garde que els mots uniques, on ne compte pas les repetitions
fuzz.token_set_ratio("plus jamais ça !", "plus jamais Jamais JAMAIS ça !!!")

Chargeons à présent le fichier `all.txt` (tronqué) : 

In [ ]:
data_path = "../data/all.txt"
LIMIT = 50000000
text = open(data_path, "r", encoding="utf-8").read()[:LIMIT]

tokens = nltk.wordpunct_tokenize(text)
print(f"{len(tokens)} words (tokens) found")

vocab = set(tokens)
print(f"{len(vocab)} different word forms")

Nous pouvons identifier les variantes d'un mot par ordre décroissant de similarité : 

In [ ]:
# on regarde le tokenn unique du corpus et on regarde quels sont les mots les plus proches
# quand on a 100 on voit une grande proximité avec le temre de recherche, ici Bruxelles
word = "bruxelles"
process.extract(word, vocab, limit=15)

Les caractères accentués ne sont pas pris en compte pour le calcul du score, mais cela reste une bonne base pour récupérer les occurrences mal OCRisées d'un mot.